<h1>Visualisation de données : Telecom Churn Prediction</h1>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from scipy import stats
from plotly.offline import iplot, init_notebook_mode

In [ ]:
sns.set(style='white')
pd.set_option('display.max_columns',None)


In [ ]:
df=pd.read_excel("Telco_customer_churn.csv.xlsx")

In [ ]:
# Une fonction qui nous permet d'avoir un resumé rapide du dataset
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Nom'] = summary['index']
    summary = summary[['Nom','dtypes']]
    summary['Manquants'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['Premiere Valeur'] = df.loc[0].values
    summary['Deuxieme Valeur'] = df.loc[2].values
    summary['Avant derniere Valeur'] = df.iloc[-2].values
    summary['Derniere Valeur'] = df.iloc[-1].values

    for name in summary['Nom'].value_counts().index:
        summary.loc[summary['Nom'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=10),2)
    return summary

In [ ]:
resumetable(df)

In [ ]:
df.describe()

In [ ]:
print(df.columns.values)

<p> On constate que le "Total charges" est un string, donc on va le convertir  en int</p> 

In [ ]:
# Le total charges est un string
# donc on va le convertir en un type numeric
df['Total Charges'] = pd.to_numeric(df['Total Charges'],errors='coerce')

<p>On visualise le nombre de valeur null dans les champs</p>

In [ ]:
df.isnull().sum()

<p>On peut constater :</p> 
<ul>
    <li>
        qu'il ya 5174 personnes qui y sont encore
    </li>
    <li>
        qu'il ya 11 'Total charges' non calculés
    </li>
</ul>

<p>Le nombre de lignes contenant un 'Total charges' n'est pas considérables vu la taille des données donc on peut les supprimer :</p> 

In [ ]:
#df=df[pd.notnull(df['Total Charges'])]
df=df[df['Total Charges'].notna()]

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

<p>On peut aussi observer si certaines variables (catégoriques) auront un impact sur les resultats, en utilisant les variables fictives :</p>

In [ ]:
# je retire l'ID Custumer qui n'a pas d'impact
df2= df.iloc[:,1:]
df2=df2.drop(['Zip Code','Lat Long','Latitude','Longitude','Churn Label'],axis=1)
df3=df2
df_fic=pd.get_dummies(df2) # pour transformer toutes les variables categoriques en num
df2

In [ ]:
df2['Senior Citizen'].replace(to_replace='Yes',value=1,inplace=True)
df2['Senior Citizen'].replace(to_replace='No',value=0,inplace=True)
df2['Dependents'].replace(to_replace='Yes',value=1,inplace=True)
df2['Dependents'].replace(to_replace='No',value=0,inplace=True)
df2['Multiple Lines'].replace(to_replace='Yes',value=1,inplace=True)
df2['Multiple Lines'].replace(to_replace='No',value=0,inplace=True)
df2.tail()

In [ ]:
corr_data=pd.DataFrame(df2)
mask=np.zeros_like(corr_data)
plt.figure(figsize=(10,7))
sns.heatmap(corr_data.corr(),annot=True,linewidths=2)
#df_fic.corr()['Churn Value'].sort_values(ascending = False).plot(kind='bar')

<center><h1> Exploration Proprement Dite </h1></center>

<p>Afin de mieux comprendre la tendance de nos données, il sera efficace pour nous de les explorer 
en visualisant les différentes tendances des variables ( de facon individuelles, en tranches puis en dès). 
Tout ceci pour se faire de probables hypothèses</p>

<b>I. Démographie<b/>

<b>I.1 Le sexe

In [ ]:
colors=['#4D3425','#E4512B']
ax=(df2['Gender'].value_counts()*100.0/len(df2)).plot(kind='bar',stacked=True,rot=0,color=colors)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel("% Clients")
ax.set_xlabel("Sexe")
ax.set_ylabel("% Clients")
ax.set_title("Distribution Par Sexe")


#Collection des données dans une liste pour l'affichage
#dans les barres
totals=[]

for i in ax.patches:
    totals.append(i.get_width())

total = sum(totals)

for i in ax.patches:
    ax.text(i.get_x()+.15,i.get_height()-3.5,\
            str(round((i.get_height()/total),1))+'%',
            fontsize=12,
            color='white',
            weight = 'bold'
           )
    

Ainsi on peut noter qu'environ la moitiée de notre DataSet est constituée d'hommes et l'autre moitiée de femmes

<b>I.2 Les personnes agées

In [ ]:
ax=(df3['Senior Citizen'].value_counts()*100.0/len(df3))\
.plot.pie(autopct='%.1f%%',labels=['No','Yes'],figsize=(5,5),fontsize=12)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('Personnes agées',fontsize = 12)
ax.set_title('% des personnes agées', fontsize = 12)

16,2% des clients est très agée, donc la plupart des personnes de notre dataset est jeune

<b>I.3 Statut de partenaires et de personnes à charge

In [ ]:
df4=pd.melt(df,id_vars=['CustomerID'],value_vars=['Dependents','Partner'])
df5=df4.groupby(['variable','value']).count().unstack()
df5=df5*100/len(df)
colors = ['#4D3430','#E4512B']
ax=df5.loc[:,'CustomerID'].plot.bar(stacked=True,color=colors,figsize=(8,6),rot=0,width=.2)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('% Clients',size = 14)
ax.set_xlabel('')
ax.set_title('% Clients ayant des partenaires ou des personnes à charges',size = 14)
ax.legend(loc = 'center',prop={'size':14})

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0f}%'.format(height), (p.get_x()+.25*width, p.get_y()+.4*height),
                color = 'white',
               weight = 'bold',
               size = 14)

De cette figure on peut conclure que:
<ul>
    <li>23% des clients ont des personnes à charges</li>
    <li>48% des clients ont des partenaires</li>
</ul>

Il serait aussi interessant de visualiser les clients ayant à la fois des partenaires et des personnes à charge

In [ ]:
partner_dependents = df.groupby(['Partner','Dependents']).size().unstack()

colors = ['#4D3430','#E4512B'] 
ax = (partner_dependents.T*100.0/partner_dependents.T.sum()).T.plot(kind='bar',width=0.2,stacked=True,rot=0,figsize=(8,6),color=colors)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.legend(loc='center',prop={'size':14},title = 'Personnes à charges',fontsize =14)
ax.set_ylabel('% Clients',size = 14)
ax.set_title("% Clients avec/sans personnes à charge selon qu'ils ont ou non un partenaire",size = 14)
ax.xaxis.label.set_size(14)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0f}%'.format(height), (p.get_x()+.25*width, p.get_y()+.4*height),
                color = 'white',
               weight = 'bold',
               size = 14)

On peut constater que :  
<ul>
    <li>La grande majorité (92%) des personnes n'ayant pas de partenaires n'ont pas de personnes à charge</li>
    <li>40% des personnes ayant un partenaire a une personne à charge</li>
</ul>

<b>II. Information sur les comptes clients<b/>

<b>II.1 Selon la durée d'abonnement 

In [ ]:
ax= sns.distplot(df['Tenure Months'],hist=True,kde=False,color='darkblue',
                 hist_kws={'edgecolor':'black'},
                kde_kws={'linewidth':6})
ax.set_ylabel('Nombre de Clients')
ax.set_xlabel("Durée d'abonnements(en mois)")
ax.set_title("Nombre de Clients par durée d'abonnements")

De cet histogramme nous pouvons constater:
<ul>
    <li>Q'un grand nombre (>1200) de clients vient de s'abonner à la socièté</li>
    <li>Q'un grand nombre (entre 800 et 100) de clients s'est abonné à la socièté il ya environ 70 mois</li>
</ul>

Cela peut s'expliquer à cause du grand nombre de service. Et donc il ya des services interessants qui retiennent les clients et d'autres non

<b>II.1 Selon le contrat

In [ ]:
ax = df['Contract'].value_counts().plot(kind = 'bar',rot = 0, width = 0.5)
ax.set_ylabel('Nombre de clients')
ax.set_title('Nombre de clients par type de contrat')

De cet histogramme on peut noter que 
<ul>
    <li>la grande majorité des clients a un contrat de paie mensuel</li>
    <li> Il ya un nombre presque égale entre les abonnés à contrat annuel et biannuel</li>
</ul>


Il serait interessant de visualiser la durée du contrat par type d'abonnements

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=3, ncols=1, sharey = True, figsize = (10,15))

ax = sns.distplot(df[df['Contract']=='Month-to-month']['Tenure Months'],
                   hist=True, kde=False, 
                  color = 'turquoise',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax1)
ax.set_ylabel('Nombre de clients')
ax.set_xlabel("Durée d'abonnements (en mois)")
ax.set_title("Contrat Paiement Mensuel")

ax = sns.distplot(df[df['Contract']=='One year']['Tenure Months'],
                   hist=True, kde=False,
                  color = 'steelblue',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax2)
ax.set_xlabel('Durée du contrat (mois)',size = 14)
ax.set_title('Contrat Paiement Annuel',size = 14)

ax = sns.distplot(df[df['Contract']=='Two year']['Tenure Months'],
                   hist=True, kde=False,
                  color = 'darkblue',
                   hist_kws={'edgecolor':'black'},
                   kde_kws={'linewidth': 4},
                 ax=ax3)

ax.set_xlabel("Durée d'abonnements(mois)")
ax.set_title('Contrat paiement biannuel')

De ces graphes nous constatons que :
<ul>
    <li>Les contrats avec paiement Mensuel dure environ 2 mois</li>
    <li> Ceux qui sont le plus fidèle sont les clients ayant souscrit à des services avec contrat à paiement Biannule  avec une durée de plus de 70 mois </li>
</ul>

<b>III. Information sur la repartition des services<b/>

In [ ]:
df.columns.values

In [ ]:
services = ['Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies']
fig,axes = plt.subplots(nrows=3,ncols=3,figsize=(15,12))
for i,item in enumerate(services):
    if i<3:
        ax=df[item].value_counts().plot(kind='bar',ax=axes[i,0],rot=0)
    elif i>=3 and i <6:
        ax = df[item].value_counts().plot(kind = 'bar',ax=axes[i-3,1],rot = 0)
    elif i<9:
        ax = df[item].value_counts().plot(kind = 'bar',ax=axes[i-6,2],rot = 0)
    ax.set_title(item)

On peut ainsi faire une liste des services les plus utilisées :
<ul type="square">
    <li>Phones services</li>
    <li> Streaming Service</li>
    <li> Streaming Movies</li>
    <li> Internet (Fibre optique)</li>
</ul>

<b>IV. Relation entre les frais mensuels et les frais totaux<b/>

In [ ]:
df[['Monthly Charges','Total Charges']].plot.scatter(x='Monthly Charges',y='Total Charges')

De cette figure nous constatons que le total des frais augmente au fur et à mesure que la facture mensuelle d'un client augmente.

<b>V. Une analyse sur l'étiquettte (churn)<b/>

<b>V.1 Taux de résiliation<b/>

In [ ]:
colors = ['#4D3425','#E4512B']
ax = (df['Churn Label'].value_counts()*100.0/len(df)).plot(kind='bar',
                                                      stacked = True,
                                                      rot = 0,
                                                      color = colors,
                                                      figsize = (8,6))
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('% Clients',size = 14)
ax.set_xlabel('Désabonnements(Churn)',size = 14)
ax.set_title('Taux de désabonnement', size = 14)


#On affiche le taux sur chaque barre de l'histogramme
totals = []

for i in ax.patches:
    totals.append(i.get_width())


total = sum(totals)

for i in ax.patches:
    ax.text(i.get_x()+.15, i.get_height()-4.0, \
            str(round((i.get_height()/total), 1))+'%',
            fontsize=12,
            color='white',
           weight = 'bold',
           size = 14)

Ainsi:
<ul type="square">
    <li>73.4% des clients ne sont pas désabonnés</li>
    <li>26.4% des clients se sont désabonnées</li>
</ul>

<b>V.2 Désabonnement & Durée d'abonnements<b/>

In [ ]:
sns.boxplot(x = df['Churn Label'], y = df['Tenure Months']).set(xlabel='Désabonnement',ylabel="Durée d'abonnement (mois)")

Ceux qui résilient leur contrat sont pour la majorité ceux qui ne durent pas dans l'entreprise

<b>V.3 Désabonnement & la catégorie d'age<b/>

In [ ]:
colors = ['#4D3425','#E4512B']
seniority_churn = df.groupby(['Senior Citizen','Churn Label']).size().unstack()

ax=(seniority_churn.T*100.0/seniority_churn.T.sum()).T.plot(kind='bar',
                                                            width=0.2,
                                                            stacked=True,
                                                            rot=0,
                                                            figsize=(8,5),
                                                            color=colors
                                                           )
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.legend(loc='center',prop={'size':14},title='Désabonnement')
ax.set_ylabel('% Clients')
ax.set_title("Désabonnement par catégorie d'age(Personne agée)",size = 14)

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate('{:.0f}%'.format(height), (p.get_x()+.25*width, p.get_y()+.4*height),
                color = 'white',
               weight = 'bold',size =14)

Les clients qui resilient plus leur contrat sont les personnes les agées ( 2 fois plus elevé que les jeunes)

<b>V.4 Désabonnement, charges Mensuelles & Quelques services<b/>

In [ ]:
color_op = ['#5527A0', '#BB93D7', '#834CF7', '#6C941E', '#93EAEA', '#7425FF', '#F2098A', '#7E87AC', 
            '#EBE36F', '#7FD394', '#49C35D', '#3058EE', '#44FDCF', '#A38F85', '#C4CEE0', '#B63A05', 
            '#4856BF', '#F0DB1B', '#9FDBD9', '#B123AC']

def PieChart(df_cat, df_value, title, limit=15):
    tmp_churn = df[df['Churn Value'] == 1].groupby(df_cat)[df_value].sum().nlargest(limit).to_frame().reset_index()
    tmp_no_churn = df[df['Churn Value'] == 0].groupby(df_cat)[df_value].sum().nlargest(limit).to_frame().reset_index()

    trace1 = go.Pie(labels=tmp_no_churn[df_cat], 
                    values=tmp_no_churn[df_value], name= "No-Churn", hole= .5, 
                    hoverinfo="label+percent+name+value", showlegend=True,
                    domain= {'x': [0, .48]})

    trace2 = go.Pie(labels=tmp_churn[df_cat], 
                    values=tmp_churn[df_value], name="Churn", hole= .5, 
                    hoverinfo="label+percent+name+value", showlegend=False, 
                    domain= {'x': [.52, 1]})

    layout = dict(title= title, height=450, font=dict(size=15),
                  annotations = [
                      dict(
                          x=.20, y=.5,
                          text='No Churn', 
                          showarrow=False,
                          font=dict(size=20)
                      ),
                      dict(
                          x=.80, y=.5,
                          text='Churn', 
                          showarrow=False,
                          font=dict(size=20)
                      )
        ])

    fig = dict(data=[trace1, trace2], layout=layout)
    iplot(fig)

<b>V.5.1 Internets<b/>

In [ ]:
no_churn_monthly_renenue = df['Monthly Charges'].sum()
PieChart("Internet Service", 'Monthly Charges', "Désabonnement Par service Internet", limit=10)

La majorité des clients souscrivent plus au service de Fibre optique et se désabonnent aussi le plus de ce service

<b>V.5.2 Types de contrats<b/>

In [ ]:
PieChart("Contract", 'Monthly Charges', "Désabonnement par type de contrat avec pourcentage de frais mensuelles", limit=10)

<b>V.5.3 Multiples Lines<b/>

In [ ]:
PieChart("Multiple Lines", 'Monthly Charges', "Désabonnement par Types de Lignes", limit=10)

<b>V.5.3 Protection de l'appareil<b/>

In [ ]:
PieChart("Device Protection", 'Monthly Charges', "Désabonnement de client ayant souscrit à un service internet avec protection de l'appareil", limit=10)


Un résumé général sur les services

In [ ]:

df.loc[:,'Engaged'] = np.where(df['Contract'] != 'Month-to-month', 1,0)
df.loc[:,'YandNotE'] = np.where((df['Senior Citizen']==0) & (df['Engaged']==0), 1,0)
df.loc[:,'ElectCheck'] = np.where((df['Payment Method'] == 'Electronic check') & (df['Engaged']==0), 1,0)
df.loc[:,'fiberopt'] = np.where((df['Internet Service'] != 'Fiber optic'), 1,0)
df.loc[:,'StreamNoInt'] = np.where((df['Streaming TV'] != 'No internet service'), 1,0)
df.loc[:,'NoProt'] = np.where((df['Online Backup'] != 'No') |\
                                    (df['Device Protection'] != 'No') |\
                                    (df['Tech Support'] != 'No'), 1,0)

df['TotalServices'] = (df[['Phone Service', 'Internet Service', 'Online Security',
                                       'Online Backup', 'Device Protection', 'Tech Support',
                                       'Streaming TV', 'Streaming Movies']]== 'Yes').sum(axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#encodage de l'etiquette
le = LabelEncoder()

tmp_churn = df[df['Churn Value'] == 1]
tmp_no_churn = df[df['Churn Value'] == 0]

bi_cs = df.nunique()[df.nunique() == 2].keys()
dat_rad = df[bi_cs]

for cols in bi_cs :
    tmp_churn[cols] = le.fit_transform(tmp_churn[cols])
    

data_frame_x = tmp_churn[bi_cs].sum().reset_index()
data_frame_x.columns  = ["feature","yes"]
data_frame_x["no"]    = tmp_churn.shape[0]  - data_frame_x["yes"]
data_frame_x  = data_frame_x[data_frame_x["feature"] != "Churn"]

#nombre de 1 (oui)
trace1 = go.Scatterpolar(r = data_frame_x["yes"].values.tolist(), 
                         theta = data_frame_x["feature"].tolist(),
                         fill  = "toself",name = "Churn 1's",
                         mode = "markers+lines", visible=True,
                         marker = dict(size = 5)
                        )

#nombre de 0 (non)
trace2 = go.Scatterpolar(r = data_frame_x["no"].values.tolist(),
                         theta = data_frame_x["feature"].tolist(),
                         fill  = "toself",name = "Churn 0's",
                         mode = "markers+lines", visible=True,
                         marker = dict(size = 5)
                        ) 
for cols in bi_cs :
    tmp_no_churn[cols] = le.fit_transform(tmp_no_churn[cols])
    
data_frame_x = tmp_no_churn[bi_cs].sum().reset_index()
data_frame_x.columns  = ["feature","yes"]
data_frame_x["no"]    = tmp_no_churn.shape[0]  - data_frame_x["yes"]
data_frame_x  = data_frame_x[data_frame_x["feature"] != "Churn"]

#nombre de 1(oui)
trace3 = go.Scatterpolar(r = data_frame_x["yes"].values.tolist(),
                         theta = data_frame_x["feature"].tolist(),
                         fill  = "toself",name = "NoChurn 1's",
                         mode = "markers+lines", visible=False,
                         marker = dict(size = 5)
                        )

#nombre de 0(non)
trace4 = go.Scatterpolar(r = data_frame_x["no"].values.tolist(),
                         theta = data_frame_x["feature"].tolist(),
                         fill  = "toself",name = "NoChurn 0's",
                         mode = "markers+lines", visible=False,
                         marker = dict(size = 5)
                        ) 

data = [trace1, trace2, trace3, trace4]

updatemenus = list([
    dict(active=0,
         x=-0.15,
         buttons=list([  
            dict(
                label = 'Dist Désabonné',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]}, 
                     {'title': "Repartition du nombre de Clients s'étant désabonné par service"}]),
             
             dict(
                  label = 'Dest Non-Désabonné',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]},
                     {'title': "Repartition du nombre de Clients ne s'étant pas désabonné par service"}]),

        ]),
    )
])

layout = dict(title='ScatterPolar de désabonnement des Clients', 
              showlegend=False,
              updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

iplot(fig)

<b>V.6 Désabonnement & les charges mensuelles<b/>

In [ ]:
ax=sns.kdeplot(df['Monthly Charges'][(df['Churn Label']=='No')],
              color='Red',shade=True)
ax=sns.kdeplot(df['Monthly Charges'][(df['Churn Label']=='Yes')],
              color='Blue',shade=True)
ax.legend(["Abonné","Désabonné"],loc='upper right')
ax.set_ylabel('Densité')
ax.set_xlabel('Charges Mensuelles')
ax.set_title('Distribution de désabonnement par charge Mensuel')

On peut voir que <b>plus la charge mensuelle augmente, plus les clients se désabonnent

<b>V.6 Désabonnement & les charges Totales<b/>

In [ ]:
ax=sns.kdeplot(df['Total Charges'][(df['Churn Label']=='No')],
              color='Red',shade=True)
ax=sns.kdeplot(df['Total Charges'][(df['Churn Label']=='Yes')],
              color='Blue',shade=True)
ax.legend(["Abonné","Désabonné"],loc='upper right')
ax.set_ylabel('Densité')
ax.set_xlabel('Charges Mensuelles')
ax.set_title('Distribution de désabonnement par charge Mensuel')

Le graphe laisse voir que <b>plus le taux de charges totales augmentent, moins il ya de résilitation

<center><h1> Quelques Algorithmes Prédictifs </h1></center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2
import seaborn as sns
import operator
from tabulate import tabulate

In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split,learning_curve,GridSearchCV,cross_val_score
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from IPython.display import display, HTML
from sklearn.naive_bayes import ComplementNB,GaussianNB,MultinomialNB,BernoulliNB,CategoricalNB
from sklearn.metrics import auc,confusion_matrix,precision_score,precision_recall_fscore_support ,f1_score,consensus_score,recall_score,accuracy_score,plot_confusion_matrix,classification_report,precision_recall_curve,roc_auc_score,roc_curve
from xgboost import XGBClassifier # à installer
from lightgbm import LGBMClassifier # à installer

In [ ]:
pd.set_option("Display.max_columns",100)
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
df= pd.read_excel("Telco_customer_churn.csv.xlsx")

encodeur = LabelEncoder()
scaler=StandardScaler()

<h1>Nettoyage</h1>

In [ ]:
df["Total Charges"]=pd.to_numeric(df["Total Charges"],errors="coerce")
df["Total Charges"]=df["Total Charges"].astype(float)
df=df[df["Total Charges"].notna()]


In [ ]:
print("il y a {} valeur nulle".format(df.iloc[:,:-1].isnull().sum().sum()) )


<h1>Fonctions utiles</h1>

In [ ]:
# pour evaluer une liste d'algorithme
def evaluation(x_train,y_train,x_test,y_test,modelList={},confusion=False,roc_cu=True,learning_cu=False):
    areas={}
    models={}
    if len(modelList)!=0:
        if roc_cu:
            plt.figure(figsize=(12,8))
            plt.plot([0,1],[0,1],"r")
        for k,model in modelList.items():
            model.fit(x_train,y_train)
            models[k]=model
            if roc_cu:
                probs = model.predict_proba(x_test)[:,1]
                fpr , tpr , thresholds = roc_curve(y_test,probs)
                areas[k]=roc_auc_score(y_test,probs)
                plt.plot(fpr,tpr,"",label=k)
        if roc_cu:
            plt.legend()
            plt.show()
    
    areas = dict(sorted(areas.items(),key=operator.itemgetter(1),reverse=True))
    
    for k,area in areas.items():
        print(f"l aire du modele {k} est {area}")
    
    if confusion:
        for k,model in models.items():
            plt.figure()
            y_pred = model.predict(x_test)
            print(f"modele {k}")
            plot_confusion_matrix(model,x_test,y_test,display_labels=["No churn","Churn"])
            print(classification_report(y_test,y_pred,digits=6,target_names=["No churn","Churn"]))
            plt.show()
    
    if learning_cu:
        for k,model in models.items():
            N,train_score,val_score_train=learning_curve(model,x_train,y_train,cv=5,scoring="f1",train_sizes=np.linspace(0.1,1,10))
            #N,test_score,val_score_test=learning_curve(model,x_test,y_test,cv=5,scoring="f1",train_sizes=np.linspace(0.1,1,10))
            plt.figure(figsize=(12,8))
            plt.plot(N,train_score.mean(axis=1),label="Train score")
            plt.plot(N,val_score_train.mean(axis=1),label="validation score train")
            #plt.plot(N,test_score.mean(axis=1),label="Test score")
            #plt.plot(N,val_score_test.mean(axis=1),label="validation score test")
            plt.title(f"Modèle {k} ")
            plt.legend()
    return models
            
            
def featuresImportances(model,index,nb_features="None"):
    plt.figure(figsize=(14,5))
    serie=pd.Series(model.feature_importances_,index=index).sort_values(ascending=False)
    if nb_features=="None":
        serie.plot.bar(log=True)
    else:
        serie[:nb_features].plot.bar(log=True)
        
def final_model(model,X,seuil=0):
    return model.decision_function(X) > seuil

def comparaison_models(models,tests_set={}):
    arrays=[
        np.array([]),np.array([])
    ]
    d=np.array([])
    for nom,model in models.items():
        arrays[0]=np.append(arrays[0],[nom,nom])
        arrays[1]=np.append(arrays[1],["No churn","Churn"])

    for name,model in models.items():
        x_test,y_test=tests_set.get(name)[0],tests_set.get(name)[1]
        y_pred = model.predict(x_test)
        res=precision_recall_fscore_support(y_test,y_pred)
        accuracy=accuracy_score(y_test,y_pred)
        res=list(res)
        res.insert(3,np.array([accuracy,accuracy]))
        res=tuple(res)
        d=np.append(d,np.array([val[0]for val in res]))
        d=np.append(d,np.array([val[1]for val in res]))

    d=d.reshape(len(models)*2,5)
    arrays=np.array(arrays)
    data=pd.DataFrame(d,columns=["précison","recall","f1_score","accuracy","test_size"])
    data.iloc[:,:-1]=data.iloc[:,:-1].apply(lambda x:x*100)
    data["Modèles"]=arrays[0,:]
    data["Etat"]=arrays[1,:]
    data=data.set_index(['Modèles'])
    print(tabulate(data, tablefmt='grid',headers="keys",showindex="always"))
    #data=data.set_index(['Modèles',"accuracy",'Etat'])
    #display(HTML(data.to_html()))
    
def precision_recall_f1(model):
    y_pred=model.predict(x_test)
    res=precision_recall_fscore_support(y_test,y_pred)
    data=pd.DataFrame(res,index=["précison","recall","f1_score","test_size"],columns=["No churn","Churn"])
    sns.heatmap(data.iloc[:-1,:].apply(lambda x:x*100),vmin=0,annot=True,fmt=".6g",vmax=100)

<center><h1>NAIVES BAYES</h1></center>

In [ ]:
df1=df

df1['Gender'] = df1['Gender'].map({'Male': 1, 'Female': 0})
df1['Streaming Movies'] = df1['Streaming Movies'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Streaming TV'] = df1['Streaming TV'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Tech Support'] = df1['Tech Support'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Device Protection'] = df1['Device Protection'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Online Backup'] = df1['Online Backup'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Online Security'] = df1['Online Security'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Contract'] = df1['Contract'].map({'Month-to-month': 1, 'One year': 0, 'Two year':2})
df1['Internet Service'] = df1['Internet Service'].map({'Fiber optic': 1, 'No': 0, 'DSL':2})
df1['Total Charges'] = pd.to_numeric(df1['Total Charges'],errors='coerce')
df1['Multiple Lines'] = df1['Multiple Lines'].map({'Yes': 1, 'No': 0, 'No phone service':0})
df1['Phone Service'] = df1['Phone Service'].map({'Yes': 1, 'No': 0, 'No phone service':0})
df1['Dependents'] = df1['Dependents'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Partner'] = df1['Partner'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Senior Citizen'] = df1['Senior Citizen'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Paperless Billing'] = df1['Paperless Billing'].map({'Yes': 1, 'No': 0, 'No internet service':0})
df1['Payment Method'] = df1['Payment Method'].map({'Mailed check': 1, 'Electronic check': 0, 'Bank transfer (automatic)':2, 'Credit card (automatic)': 3,})
df1=df1.drop(['CustomerID','Count','Country','State','City','Zip Code','Lat Long','Latitude','Longitude','Churn Label','Churn Reason'],axis=1)
df_fic=pd.get_dummies(df1) # pour transformer toutes les variables categoriques en num


In [ ]:
#transformer les catégories de chaque caractéristique comme des valeurs numériques
#On fait des changements a notre dataset en changeant toutes les donées en nombres
df_naives=df1

for col in df_naives.columns:
    df_naives[col] = encodeur.fit_transform(df_naives[col])
    
#séparer les entrées (caractéristiques) et la sortie (classe)
#On crée X qui contient nos données et Y qui contient les classes
y = df_naives['Churn Value']  #les résulats (classes)
X = df_naives.drop(['Churn Value'], axis =1) #les caractéristiques

X_trainN, X_testN, y_trainN, y_testN = train_test_split(X, y, test_size=0.2, random_state=0,stratify=y)

In [ ]:
#Cette fonction est essentiel pour le naive bayes classifier vu qu'il présente plusieurs méthodes : Gaussien Multinomial et Bernoulli. 
#On va calculer le score de chaqu'une de ces méthodes .

nb = {'gaussian': GaussianNB(),
      'bernoulli': BernoulliNB(),
      'multinomial': MultinomialNB()}
scores = {}
for key, model in nb.items():
    s = cross_val_score(model, X_trainN, y_trainN, cv=5, scoring='accuracy')
    scores[key] = np.mean(s)
scores

In [ ]:
# modele à retenir celon celui qui a le meilleur score
modele = GaussianNB()

NaiveBayes=evaluation(X_trainN,y_trainN,X_testN,y_testN,modelList={"NaiveBayes":modele},confusion=True,roc_cu=False,learning_cu=False)

<center><h1>REGRESSION LOGISTIQUE</h1></center>

La régression logistique est utilisée pour le classement et pas la régression.
Mais, elle est considéré comme une méthode de classification puisqu'elle sert à estimer la probabilité d'appartenir à une classe.
Il y a trois types de régression logistique:
- **Régression logistique binaire**: ici, le but de la classification est d'identifier si un échantillon appartient à une classe ou non.
- **Régression logistique multinomiale**: ici, le but de la classification est d'identifier à quelle classe appartient-t-il un échantillon parmi plusieurs classes.
- **Régression logistique ordinale**: ici, le but de la classification est de chercher la classe d'un échantillon parmi des classes ordonnées. Un exemple de classes: non satisfait, satisfait, très sataisfait.

In [ ]:
X = df1.drop("Churn Value", axis=1)
y = df1["Churn Value"]
X_trainL, X_testL, y_trainL, y_testL = train_test_split(X, y, test_size=0.2, random_state=0,stratify=y)

X_trainL = scaler.fit_transform(X_trainL)
X_testL = scaler.transform(X_testL)

logreg = LogisticRegression(random_state=0)

regressionL=evaluation(X_trainL,y_trainL,X_testL,y_testL,modelList={"Regression Logistique":logreg},confusion=True,roc_cu=True,learning_cu=False)

In [ ]:
param =  {'penalty': ['l1', 'l2', 'elasticnet', 'none'],
          'C' : np.linspace(0,10, 100),
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'max_iter': [100,1000, 2000, 3000]}

log = LogisticRegression(random_state=0)
grid = GridSearchCV(log, param_grid=param, cv =5, n_jobs=-1)
#grid.fit(X_train, y_train)
#grid.best_params_

<h4>best parameters : {'C': 0.20202020202020202,
 'max_iter': 100,
 'penalty': 'l2',
 'solver': 'liblinear'}</h4>

In [ ]:
final_modelL = LogisticRegression(random_state=0, C=0.20202, penalty = 'l2', solver='liblinear', tol=0.2)

regressionL=evaluation(X_trainL,y_trainL,X_testL,y_testL,modelList={"Regression Logistique":final_modelL},confusion=True,roc_cu=False,learning_cu=False)

<center><h1>RANDOM FOREST CLASSIFIER</h1></center>

In [ ]:
# Drop useless columns 
dfR = df.drop(['Count','Country','State','CustomerID','Lat Long','Churn Label'],axis=1)
#One Hot Encoing using get_dummies method 
dfR = pd.get_dummies(dfR, columns = ['Contract','Dependents','Device Protection','Gender',
                                   'Internet Service','Multiple Lines','Online Backup',
                                   'Online Security','Paperless Billing','Partner',
                                   'Payment Method','Phone Service','Senior Citizen',
                                   'Streaming Movies','Streaming TV','Tech Support','City','Churn Reason'])

In [ ]:
#Create Feature variable X and target variable y
y = dfR['Churn Value']
X = dfR.drop(['Churn Value'], axis =1)

X_trainR, X_testR,y_trainR,y_testR = train_test_split(X, y, test_size = 0.20, random_state = 0,stratify = y)

rfmodel = RandomForestClassifier(n_estimators=100 , criterion='gini',random_state=0) # par defaut au prémier lieux
                                                                                        #justifier les choix des paramètres
randomF=evaluation(X_trainR,y_trainR,X_testR,y_testR,modelList={"Random Forest":rfmodel},confusion=True,roc_cu=False,learning_cu=False)

<center><h1>Famille de Gradient Boosting</h1></center>

In [ ]:
df= pd.read_excel("Telco_customer_churn.csv.xlsx")
df["Total Charges"]=pd.to_numeric(df["Total Charges"],errors="coerce")
df["Total Charges"]=df["Total Charges"].astype(float)
df=df[df["Total Charges"].notna()]

<h2>Features engineering</h2>
<ul>
    <li><h3>Création de nouvelles variables dans le but d'ameliorer la prédiction </h3></li>
    <p><u>NB</u> ces variables créées ont découlé de la compréhension du dataset</p>
</ul>

In [ ]:
#service_cols=["Phone Service","Multiple Lines","Internet Service","Online Security","Online Backup","Device Protection","Tech Support","Streaming TV","Streaming Movies","Paperless Billing"]

service_cols=['Phone Service', 'Internet Service', 'Online Security','Online Backup', 'Device Protection', 'Tech Support','Streaming TV', 'Streaming Movies']

execptions=["Gender",	"Senior Citizen",	"Partner",	"Dependents",	"Phone Service"	,"Paperless Billing",	 "Churn Label","Churn Score",	"Engaged",	"ElectCheck",	"fiberopt",	"StreamNoInt",	"NoProt"] 

df.loc[:,'Engaged'] = np.where(df['Contract'] != 'Month-to-month', 1,0) # client avec un contrat month-to-month
df.loc[:,'YandNotE'] = np.where((df['Senior Citizen']==0) & (df['Engaged']==0), 1,0) # si client est jeune et n'a pas souscrit a un abonnement month-to-month
df.loc[:,'ElectCheck'] = np.where((df['Payment Method'] == 'Electronic check') & (df['Engaged']==0), 1,0) #si le client a paye par chèque electronique et n'a pas souscrit à un payement mensuel
df.loc[:,'fiberopt'] = np.where((df['Internet Service'] != 'Fiber optic'), 1,0) # si le client utilise internet par fibre optique
df.loc[:,'StreamNoInt'] = np.where((df['Streaming TV'] != 'No internet service'), 1,0) # si le client utilise le streaming TV sans internet
df.loc[:,'NoProt'] = np.where((df['Online Backup'] != 'No') | (df['Device Protection'] != 'No') | (df['Tech Support'] != 'No'), 1,0) # si le client n'utilise pas au moins un service supplémentaire au service internet

df['nb_subscriptions'] = (df[service_cols]== 'Yes').sum(axis=1) # le nombre de service auxquels le client a souscrit

target_col=["Churn Label"]
cat_col=[]
binary_col=[]
multi_cat_col=[]
delete_col=["Churn Value","Count","Zip Code",'City','Zip Code','Lat Long','Latitude','Longitude',"CustomerID","Churn Reason"]

cat_col=df.nunique()[df.nunique() < 10].keys().tolist()
cat_col=[x for x in cat_col if x not in delete_col]
binary_col = df.nunique()[df.nunique() == 2].keys().tolist()
multi_cat_col=[x for x in cat_col if x not in binary_col]



In [ ]:
print(delete_col)
df=df.drop(delete_col,axis=1,errors="ignore")

<ul>
    <li><h3>Transformation des colones multicatégories en unicatégorie</h3></li>
</ul>

In [ ]:
df=pd.get_dummies(data=df,columns=multi_cat_col)    


<h2>Encodage</h2>

In [ ]:
#col_to_standardScale=[col for col in features.columns if col not in execptions]
#col_to_encode=[col for col in binary_col if col not in delete_col]
encoder = LabelEncoder()
for col in binary_col:
    if col not in delete_col:
        df[col]=encoder.fit_transform(df[col])



In [ ]:
features=df.drop(target_col,axis=1)
target=df[target_col].values.reshape(features.shape[0])

<h2>SelectKBest</h2>

In [ ]:
selector=SelectKBest(score_func=chi2,k=features.shape[1])
fit=selector.fit_transform(features,target)
dd=pd.DataFrame(fit.sum(axis=0))
dd.index =features.columns
dd.sort_values(dd.columns[0],ascending=False).iloc[:10].plot.bar(log=True)

<h2>Normalisation</h2>

In [ ]:
cols=[col for col in features.columns if col not in execptions]
scaler=StandardScaler() 
data=scaler.fit_transform(df[cols])
right=pd.DataFrame(data,columns=cols)
left=pd.DataFrame(df[execptions],columns=execptions)
data=left.merge(right, left_index=True, right_index=True,how="left")


In [ ]:
data.dropna(axis=0,inplace=True)
features=data.drop(target_col,axis=1)
target=data[target_col].values.reshape(features.shape[0])


<h1>Division train_set et test_set</h1>

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=0,stratify=target)


# Comparaison des modèles avec les paramètres de base

<h3>Les modèles sont classés de facon décroissante de performance par la courbe ROC </h3>
<p>Les 3 modèles sont :</p>
<ul>
    <li>LGBMClassifier avec une aire de 0.9975258951124759</li>
    <li>XGBClassifier avec une aire de 0.9971576321208317</li>
    <li>GradientBoostingClassifier avec une aire de 0.9969670171216356</li>
</ul>

<h3>La courbe ROC est suivie du Classification repport, Confusion metric et Learning curve de chaque modèle</h3>

In [ ]:
xgb = XGBClassifier()
gboost=GradientBoostingClassifier()
lgbm=LGBMClassifier()


In [ ]:
modelList={"XGBClassifier":xgb,"GradientBoost":gboost,"LGBMClassifier":lgbm}
modelsBase=evaluation(x_train,y_train,x_test,y_test,modelList,True,learning_cu=True)

### Example de réglage des hyper paramètres (cas du GradientBoostingClassifier)

<ul>
    <li>réglage du n_estimators</li>
</ul>

In [ ]:
paramsGB1={"n_estimators":range(290,321,10)}
model=GradientBoostingClassifier(learning_rate=0.2, min_samples_split=35,min_samples_leaf=3,max_depth=8,max_features='sqrt',subsample=0.8,random_state=0)
grid1=GridSearchCV(model, param_grid = paramsGB1, scoring='recall',n_jobs=4, cv=5)
grid1.fit(x_train,y_train)
print( grid1.best_params_, grid1.best_score_)

<ul>
    <li>réglage du max_depth et min_samples_split</li>
</ul>

In [ ]:
paramsGB1={"max_depth":range(3,6,1),'min_samples_split':range(420,451,10)}
model=GradientBoostingClassifier(learning_rate=0.2,n_estimators=310, min_samples_split=35,min_samples_leaf=3,max_features='sqrt',subsample=0.8,random_state=0)
grid1=GridSearchCV(model, param_grid = paramsGB1, scoring='recall',n_jobs=4, cv=5)
grid1.fit(x_train,y_train)
print( grid1.best_params_, grid1.best_score_)

<ul>
    <li>réglage du min_samples_leaf et max_features</li>
</ul>

In [ ]:
paramsGB1={'min_samples_leaf':range(29,40,1),'max_features':range(7,12,2)}
model=GradientBoostingClassifier(learning_rate=0.2,n_estimators=310, min_samples_split=430,max_features='sqrt',subsample=0.8,random_state=0)
grid1=GridSearchCV(model, param_grid = paramsGB1, scoring='recall',n_jobs=4, cv=5)
grid1.fit(x_train,y_train)
print( grid1.best_params_, grid1.best_score_)

<ul>
    <li>réglage du subsample</li>
</ul>

In [ ]:
paramsGB1={'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
model=GradientBoostingClassifier(learning_rate=0.2,n_estimators=310,max_depth=5,min_samples_split=430,min_samples_leaf=32,max_features=8 ,random_state=0)
grid1=GridSearchCV(model, param_grid = paramsGB1, scoring='recall',n_jobs=4, cv=5)
grid1.fit(x_train,y_train)
print( grid1.best_params_, grid1.best_score_)

<h1> Comparaison des modèles améliorés </h3>
<h3>Nous remarquons que la plupart des modèles ont besoin de plus de données pour devenir plus performant</h3>

In [ ]:
modelLGBM = LGBMClassifier(num_leaves=17,max_depth=10,learning_rate=0.1,n_estimators=32,subsample_for_bin=200000,min_split_gain=0.0,min_child_weight=0.001,min_child_samples=1,subsample=0.6,subsample_freq=0,colsample_bytree=1.0,reg_alpha=1e-08,reg_lambda=1e-08,random_state=0)
modelGB = GradientBoostingClassifier(learning_rate=0.2,n_estimators=310,max_depth=5,min_samples_split=430,min_samples_leaf=32,max_features=7 ,random_state=0)
modelRF = RandomForestClassifier(n_estimators=190,max_depth=7,min_samples_split=29,max_leaf_nodes=30,min_samples_leaf=1,max_samples=0.7,max_features=19,random_state=0)
modelXGB = XGBClassifier(n_estimators=250,learning_rate=0.1,max_depth=5,min_child_weight=0.001,gamma = 0.3,subsample=1,colsample_bytree = 0.83,reg_alpha=1e-5,scale_pos_weight=600)

In [ ]:
modelList={"XGBClassifier":modelXGB,"Gradientboost":modelGB,"LGBMClassifier":modelLGBM}
models=evaluation(x_train,y_train,x_test,y_test,modelList,True,learning_cu=True)

In [ ]:
best_model=models["LGBMClassifier"]

<h1>Features importances du modéle LGBM</h1>

In [ ]:
featuresImportances(best_model,features.columns,20)

<h1>Résultats finaux </h1>

<h2>Les 2 meilleurs modèles sont LGBMClassifier et GradientBoostingClassifier</h2>
<ul>
    <li>LGBMClassifier</li>
</ul>

In [ ]:
precision_recall_f1(best_model)

<ul>
    <li>GradientBoostingClassifier</li>
</ul>

In [ ]:
precision_recall_f1(models["Gradientboost"])

<ul>
    <li>Tableau comparatif</li>
</ul>

In [ ]:
models["Regression Logist"]=regressionL["Regression Logistique"]
models["RamdomForest"]=randomF["Random Forest"]
models["NaiveBayes"]=NaiveBayes["NaiveBayes"]
models

In [ ]:
test_set={
    "XGBClassifier":[x_test,y_test],
    "Gradientboost":[x_test,y_test],
    "LGBMClassifier":[x_test,y_test],
    "Regression Logist":[X_testL,y_testL],
    "RamdomForest":[X_testR,y_testR],
    "NaiveBayes":[X_testN,y_testN],
}
comparaison_models(models,test_set) 